In [16]:
import pandas as pd 
data = pd.read_csv('web_scrape.csv')
data = data[['0','1','2','3','4','5']]
data.columns = ['Title', 'Category', 'Desc', 'Installs', 'Number of Rating', 'Rating']

In [18]:
for j in range(0,len(data)):
    if data.Title[j][0] not in '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c': 
    #Comparing if first word of the title is non-English
        data.drop(j, inplace=True)
print(data.shape)

(13095, 6)


In [19]:
data.reset_index(inplace=True)

In [20]:
data.drop(columns=['index'],inplace=True)

Above is only used to combine all web scraping results. This is where the model starts.

In [21]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s)) 

In [23]:
data = data[['Title', 'Category', 'Desc']]

,Title,Category,Desc
0,Compass,Tools,Compass is one of the most if not the most ess...
1,Dropsend Android,Productivity,DropSend is a cloud service that lets you send...
2,Cloud Console,Tools,Google Cloud Console helps you manage your ser...
3,TapTube - Music Video Rhythm Game,Music,Do you want to experience the differentiated r...
4,Indiana DNR,Travel & Local,Indiana Department of Natural Resources and th...
...,...,...,...
13090,QuickBooks Accounting: Invoicing & Expenses,Business,"Track miles, send invoices, manage your expens..."
13091,Graphing Calculator,Tools,I believe every student in the world should ha...
13092,Hobby Farm HD Free,Casual,Jill’s dream has finally come true! She has he...
13093,SimpleMind Free - Intuitive Mind Mapping,Productivity,"Mind mapping helps you organize your thoughts,..."


In [24]:
data['key_words'] = ""

for index, row in data.iterrows():
    desc = row['Desc']
    category = row['Category']
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(desc)

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column for the corresponding movie
    row['key_words'] = list(key_words_dict_scores.keys())
    row['key_words'].append(category)
    row['key_words'] = listToString(row['key_words'])

# dropping the columns
data.drop(columns = ['Desc'], inplace = True)
data.drop(columns = ['Category'], inplace = True)

/Users/marlin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
data['key_words'][0]

'correct want right way adjust twitter e part adjusted stops moving number showing top easy simply turn series life one day always points north real compass side window method beveled edge wait south https :// different learning cover lubber line called deviation movable ring around holding notice large numbers first see remember pointing finally navigation might come handy w indicated exactly across professional navigating ). little heading never know get back influence magnetic sensor bezel arrow inside iron camping trips easier ► follow us came flat pointed long faces point moves bit www double triangle problem card settles west gammaplay learned terms front 120 android device red words counterclockwise simpler facebook steel objects suspect object several feet use n designed wrong direction really tricky ’ downside common mistakes com mean going remembers precise app hold installed actually like bracketing also end east turned notch zero directly navigate mind happening lost even s

In [26]:
data.set_index('Title', inplace = True)

In [27]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(data['key_words'])

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [28]:
cosine_sim

array([[1.        , 0.0496904 , 0.07284778, ..., 0.07100107, 0.11567035,
        0.0860167 ],
       [0.0496904 , 1.        , 0.10180787, ..., 0.02268046, 0.07607258,
        0.01131407],
       [0.07284778, 0.10180787, 1.        , ..., 0.06650062, 0.12267743,
        0.07464062],
       ...,
       [0.07100107, 0.02268046, 0.06650062, ..., 1.        , 0.0869582 ,
        0.0369516 ],
       [0.11567035, 0.07607258, 0.12267743, ..., 0.0869582 , 1.        ,
        0.11154549],
       [0.0860167 , 0.01131407, 0.07464062, ..., 0.0369516 , 0.11154549,
        1.        ]])

In [29]:
indices = pd.Series(data.index)

def recommendations(title, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommended movies
    recommended_apps = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 6 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_apps.append(list(data.index)[i])
        
    return recommended_apps

In [30]:
recommendations('QuickBooks Accounting: Invoicing & Expenses')

['Xero Accounting Software',
 'QuickBooks Self-Employed:Mileage Tracker and Taxes',
 'Invoice by Wave',
 'PayPal Business: Send Invoices and Track Sales',
 'First Horizon Mobile Banking',
 'EveryDollar: Budget Tool and Expense Tracker',
 'Invoice Maker & Billing App',
 'Invoice Maker: Estimate & Invoice App',
 'NVB Mobile',
 'Invoice 2go - Professional Business Invoice Maker']